<a href="https://colab.research.google.com/github/Jonathan-code-hub/Many-Mini-OR-Problems/blob/main/LinearProgramming/LP_Problem_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem 21: Monroe County is trying to determine where to place the county fire station. The locations of the county's four major towns are given in Figure 31. Town 1 is at ( 10,20 ); town 2 is at $(60,20)$; town 3 is at $(40,30)$; town 4 is at ( 80 , 60 ). Town 1 averages 20 fires per year; town 2, 30 fires; town 3, 40 fires; and town 4, 25 fires. The county wants to build the fire station in a location that minimizes the average distance that a fire engine must travel to respond to a fire. Since most roads run in either an east-west or a north-south direction, we assume that the fire engine can only do the same. Thus, if the fire station were located at $(30,40)$ and a fire occurred at town 4 , the fire engine would have to $\operatorname{travel}(80-30)+(60-40)=70$ miles to the fire. Use linear programming to determine where the fire station should be located. (Hint: If the fire station is to be located at the point $(x, y)$ and there is a town at the point $(a, b)$, define variables $e, w, n, s$ (east, west, north, south) that satisfy the equations $x-a=w-e$ and $y-b=n-s$. It should now be easy to obtain the correct LP formulation.)

In [1]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 60.7 MB/s eta 0:00:00


In [3]:
from pulp import *

import pulp

# Variables #
d = {1: 100, 2: 300, 3: 200}
months = [1,2,3]

revenue = {}
for i in months:
    for t in months:
        if t < i:
            continue
        delay = t - i
        if delay == 0:
            revenue[(i,t)] = 800
        elif delay == 1:
            revenue[(i,t)] = 700
        elif delay == 2:
            revenue[(i,t)] = 600

capacity_per_worker = 10
salary = 4000
hire_cost = 5000
fire_cost = 4000
W0 = 8

model = pulp.LpProblem("Ghostbusters_Profit_Max", pulp.LpMaximize)

x = {}
for i in months:
    for t in months:
        if t >= i:
            x[(i,t)] = pulp.LpVariable(f"x_{i}_{t}", lowBound=0, cat='Continuous')

W = {t: pulp.LpVariable(f"W_{t}", lowBound=0, cat='Continuous') for t in months}
H = {t: pulp.LpVariable(f"H_{t}", lowBound=0, cat='Continuous') for t in months}
F = {t: pulp.LpVariable(f"F_{t}", lowBound=0, cat='Continuous') for t in months}

# Objective Function: total revenue - salaries - hire/fire costs #
total_revenue = pulp.lpSum(revenue[(i,t)] * x[(i,t)] for (i,t) in x)
total_salary = salary * pulp.lpSum(W[t] for t in months)
total_hire_cost = hire_cost * pulp.lpSum(H[t] for t in months)
total_fire_cost = fire_cost * pulp.lpSum(F[t] for t in months)

model += (total_revenue - total_salary - total_hire_cost - total_fire_cost), "Profit"

# Constraints #
# 1) Each month's calls must be handled by end of March #
for i in months:
    model += pulp.lpSum(x[(i,t)] for t in months if t >= i) == d[i], f"Demand_satisfied_{i}"

# 2) Capacity in each month #
for t in months:
    model += pulp.lpSum(x[(i,t)] for i in months if i <= t) <= capacity_per_worker * W[t], f"Capacity_month_{t}"

# 3) Worker balance #
for t in months:
    if t == 1:
        model += W[t] == W0 + H[t] - F[t], f"Worker_balance_{t}"
    else:
        model += W[t] == W[t-1] + H[t] - F[t], f"Worker_balance_{t}"

# Solve #
solver = pulp.PULP_CBC_CMD(msg=0)  # set msg=1 to see solver output
model.solve(solver)

# Results #
print("Status:", pulp.LpStatus[model.status])
print(f"Objective (max profit) = ${pulp.value(model.objective):,.2f}\n")

for t in months:
    print(f"Month {t}: Workers W_{t} = {W[t].varValue:.2f}, H_{t} = {H[t].varValue:.2f}, F_{t} = {F[t].varValue:.2f}")
print()

for i,t in sorted(x):
    print(f"x_{i}_{t} = {x[(i,t)].varValue:.2f}  (arrived in {i}, served in {t}, revenue ${revenue[(i,t)]:.0f} each)")



Status: Optimal
Objective (max profit) = $150,000.00

Month 1: Workers W_1 = 10.00, H_1 = 2.00, F_1 = 0.00
Month 2: Workers W_2 = 25.00, H_2 = 15.00, F_2 = 0.00
Month 3: Workers W_3 = 25.00, H_3 = 0.00, F_3 = 0.00

x_1_1 = 100.00  (arrived in 1, served in 1, revenue $800 each)
x_1_2 = 0.00  (arrived in 1, served in 2, revenue $700 each)
x_1_3 = 0.00  (arrived in 1, served in 3, revenue $600 each)
x_2_2 = 250.00  (arrived in 2, served in 2, revenue $800 each)
x_2_3 = 50.00  (arrived in 2, served in 3, revenue $700 each)
x_3_3 = 200.00  (arrived in 3, served in 3, revenue $800 each)
